# Named Entity Recognition (NER)

## Summary

For Named Entity Recognition (NER), we compared two models: spaCy and Flair. For spaCy, we experimented with the 'en_core_web_sm', 'en_core_web_lg', and 'en_core_web_trf' models. The accuracy evaluation for each model can be found in the [spaCy documentation](https://spacy.io/models/en#en_core_web_sm).
|Model|F-score|
|------------|--------|
|en_core_web_sm|0.84|
|en_core_web_lg|0.86|
|en_core_web_trf|0.90|
|Flair (default)|0.93|

The data below was collected through interactions with the copilot. After testing with transfer learning, spaCy's 'en_core_web_trf' model proved to be the most accurate for Named Entity Recognition (NER). While spaCy is generally faster and more accurate, Flair also demonstrated strong accuracy compared to spaCy’s 'en_core_web_sm' and 'en_core_web_lg' models. However, Flair's processing speed was significantly slower, taking several minutes to produce results, whereas spaCy models generated output in just a few seconds.

For both speed and accuracy, spaCy's 'en_core_web_trf' is the best choice. One issue with the other two spaCy models was their tendency to misclassify unknown names as miscellaneous entities, organizations, or people, leading to inaccuracies. In contrast, 'en_core_web_trf' handled these cases more accurately.

*Note: The 'en_core_web_trf' model could not run on a local machine and was instead executed using Google Colab.*

------------------

**Named Entity Recognition (NER)** is an NLP technique that identifies and classifies named entities in text into predefined categories such as person names, organizations, locations, dates, and more. The goal is to transform unstructured text into structured data by tagging these entities, making it easier to analyze and extract meaningful information.

**Use Cases**

- `Information Extraction`: NER helps in extracting specific information from large volumes of text, such as identifying names of people, organizations, locations, dates, and more. This is particularly useful in fields like journalism and research.
- `Customer Support`: By identifying key entities in customer queries, NER can help chatbots and automated systems provide more accurate and relevant responses, improving customer service efficiency.
- `Sentiment Analysis`: NER can enhance sentiment analysis by pinpointing the entities being discussed, allowing for more precise sentiment scoring related to specific products, services, or individuals.
- `Question Answering Systems`: NER is crucial in developing systems that can understand and respond to user questions by identifying the entities involved in the query.
- `Financial Analysis`: In the finance sector, NER can be used to extract important data from financial reports, news articles, and other documents, aiding in trend analysis and risk assessment.
- `Content Recommendation`: By recognizing entities within text, NER can improve recommendation systems, suggesting relevant content based on identified entities.

**Models**
- spaCy
- Stanford's CoreNLP
- Flair

**Why is NER important?**

1. Enhanced Information Retrieval: NER helps customers quickly find relevant information by highlighting key entities in large volumes of text. This is particularly useful in fields like journalism, legal research, and customer support.
2. Improved Decision-Making: By extracting critical entities from documents, reports, and articles, NER enables customers to make informed decisions based on accurate and relevant data.
3. Automation and Efficiency: NER automates the process of identifying important entities, reducing the need for manual data extraction and saving time and resources.
4. Personalization: In customer service and marketing, NER can be used to tailor responses and recommendations based on identified entities, enhancing the customer experience.

## Import Dependencies and Load Data

In [20]:
import pandas as pd

pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', None)

# SpaCy
import spacy
from spacy import displacy

# Flair
from flair.data import Sentence
from flair.models import SequenceTagger

In [21]:
df = pd.read_csv(r"C:\Users\nene0\Desktop\Projects\greenflash\chat_data.csv", encoding_errors='ignore')

df.head()

,Chat_ID,Message_ID,Sender,Message
0,data_science_trend,0,user,What is the latest trend in data science?
1,data_science_trend,1,copilot,"Data science is evolving rapidly, and several key trends are shaping the field in 2024:\n- Generative AI: This technology continues to capture significant attention. While many organizations are excited about its potential, only a small percentage have implemented it at scale. The focus is now on integrating generative AI into existing systems and ensuring it delivers real economic value.\n- Real-time Analytics: With advancements in data capturing tools, real-time analytics is becoming more prevalent. This allows for immediate insights and decision-making based on up-to-the-minute data.\n- Deepfake Technology: The use of AI to create realistic fake videos and audio is on the rise. While it has many applications, it also poses significant risks, such as fraud and misinformation.\n- Digital Twins: These are virtual replicas of physical systems that can be used for simulation and analysis. They are becoming more sophisticated and are being used in various industries to optimize operations and predict outcomes.\n- Autonomous Data Warehouses: These self-managing data warehouses use AI to automate tasks like tuning, scaling, and security, making data management more efficient.\nThese trends highlight the growing importance of AI and real-time data in driving innovation and efficiency in various sectors. Are you particularly interested in any of these trends, or is there another aspect of data science you'd like to explore?"
2,data_science_trend,2,user,Can you tell me more about generative AI?
3,data_science_trend,3,copilot,"Generative AI is a fascinating and rapidly evolving field within artificial intelligence. Here are some key points about it:\nWhat is Generative AI?Generative AI refers to algorithms that can create new content, such as text, images, audio, and even video, based on the data they have been trained on. Unlike traditional AI, which focuses on recognizing patterns and making predictions, generative AI can produce original and creative outputs.\nKey Trends in 2024- Increased Adoption: In 2024, the adoption of generative AI has surged, with 65% of organizations regularly using it, nearly double the percentage from the previous year. This technology is being integrated into various business functions, particularly in marketing, sales, and product development.\n- Customization and Accessibility: Companies like Google and OpenAI are developing platforms that allow users to create customized AI models and chatbots without needing extensive coding skills. This makes generative AI more accessible to a broader audience.\n- Multimodal Capabilities: State-of-the-art generative AI models, such as GPT-4 and Gemini, can process not only text but also images and videos. This multimodal capability opens up new applications, such as generating detailed property descriptions from photos and videos for real estate agents.\n- Economic and Energy Considerations: As generative AI becomes more widespread, there is a focus on balancing the economic benefits with energy consumption and sustainability. Efficient use of resources is becoming a critical factor in the deployment of these technologies.\n- Trust and Security: Ensuring the accuracy and security of generative AI outputs is a major concern. Organizations are developing practices to mitigate risks such as misinformation and bias.\nApplicationsGenerative AI is being used in various fields, including:\n- Content Creation: Writing articles, creating marketing materials, and generating social media content.\n- Design: Producing artwork, designing products, and creating virtual environments.\n- Healthcare: Assisting in drug discovery and generating synthetic medical data for research.\n- Entertainment: Creating music, scripts, and even entire virtual worlds for video games and movies.\nGenerative AI is transforming how we create and interact with digital 

In [22]:
df['Chat_ID'].unique()

array(['data_science_trend', 'food_history_companies', 'gaming',
       'greek_myth', 'job_market', 'jokes', 'music_kpop', 'pets',
       'philoshophy', 'rich_poor_countries',
       'tech_product_recommendation', 'travel', 'largest_adj_product',
       'jarritos_flavors'], dtype=object)

In [23]:
travel_chat = df[df['Chat_ID']=='travel']
travel_chat.reset_index(drop=True, inplace=True)

travel_chat.head()

,Chat_ID,Message_ID,Sender,Message
0,travel,0,user,Can you recommend top 10 places to travel in East Asia where the public security is considered very safe?
1,travel,1,copilot,"Sure! Here are ten of the safest places to travel in East Asia, based on public security and overall safety:\n- Singapore: Known for its strict laws and efficient law enforcement, Singapore is often ranked as one of the safest cities in the world.\n- Japan: With low crime rates and a high standard of living, Japan is a top destination for safe travel.\n- South Korea: South Korea boasts a low crime rate and is known for its friendly and helpful locals.\n- Taiwan: Taiwan is praised for its safety, cleanliness, and friendly atmosphere.\n- Hong Kong: Despite recent political unrest, Hong Kong remains relatively safe for tourists, with low violent crime rates.\n- Malaysia: Particularly cities like Kuala Lumpur and Penang, Malaysia is known for its safety and hospitality.\n- Vietnam: Cities like Hanoi and Ho Chi Minh City are generally safe for tourists, with low levels of violent crime.\n- Mongolia: Known for its vast landscapes and nomadic culture, Mongolia is a safe destination with low crime rates.\n- Bhutan: This peaceful country is known for its low crime rates and emphasis on Gross National Happiness.\n- Brunei: With strict laws and a high standard of living, Brunei is considered very safe for travelers.\nThese destinations offer a combination of cultural richness, natural beauty, and a high level of public security. Have you visited any of these places before, or are you planning a trip soon?\n: Storyteller Travel\n: Nomad Capitalist\n: Adventure to Every Country"
2,travel,2,user,What is the most searched topic for those people who decided to travel to Taiwan?
3,travel,3,copilot,"One of the most searched topics for travelers planning a trip to Taiwan is ""things to know before visiting Taiwan"". This includes practical tips such as booking accommodation early, making restaurant reservations, using the EasyCard for public transportation, and downloading essential transportation apps.\nTravelers are also interested in Taiwan's night markets, street food, and cultural attractions like temples and historical sites. Additionally, many look up information on hiking trails and natural hot springs, as Taiwan is known for its beautiful landscapes and outdoor activities.\nAre you planning a trip to Taiwan, or just curious about what others are interested in?\n: Lonely Planet\n: The Sandy Feet"
4,travel,4,user,"I am curious to know what other people are interested in. If you are a human, out of those top 10 places, where would you want to go?"


## SpaCy

In [24]:
ner = spacy.load("en_core_web_lg")

def extract_entities(text):
    doc = ner(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

travel_chat.loc[:, 'Spacy_Lg'] = travel_chat['Message'].apply(extract_entities)

In [25]:
travel_chat.loc[:, 'Spacy_Lg_Count'] = travel_chat['Spacy_Lg'].apply(len)

In [26]:
travel_chat.head(3)

,Chat_ID,Message_ID,Sender,Message,Spacy_Lg,Spacy_Lg_Count
0,travel,0,user,Can you recommend top 10 places to travel in East Asia where the public security is considered very safe?,"[(10, CARDINAL), (East Asia, LOC)]",2
1,travel,1,copilot,"Sure! Here are ten of the safest places to travel in East Asia, based on public security and overall safety:\n- Singapore: Known for its strict laws and efficient law enforcement, Singapore is often ranked as one of the safest cities in the world.\n- Japan: With low crime rates and a high standard of living, Japan is a top destination for safe travel.\n- South Korea: South Korea boasts a low crime rate and is known for its friendly and helpful locals.\n- Taiwan: Taiwan is praised for its safety, cleanliness, and friendly atmosphere.\n- Hong Kong: Despite recent political unrest, Hong Kong remains relatively safe for tourists, with low violent crime rates.\n- Malaysia: Particularly cities like Kuala Lumpur and Penang, Malaysia is known for its safety and hospitality.\n- Vietnam: Cities like Hanoi and Ho Chi Minh City are generally safe for tourists, with low levels of violent crime.\n- Mongolia: Known for its vast landscapes and nomadic culture, Mongolia is a safe destination with low crime rates.\n- Bhutan: This peaceful country is known for its low crime rates and emphasis on Gross National Happiness.\n- Brunei: With strict laws and a high standard of living, Brunei is considered very safe for travelers.\nThese destinations offer a combination of cultural richness, natural beauty, and a high level of public security. Have you visited any of these places before, or are you planning a trip soon?\n: Storyteller Travel\n: Nomad Capitalist\n: Adventure to Every Country","[(ten, CARDINAL), (East Asia, LOC), (Singapore, GPE), (Singapore, GPE), (one, CARDINAL), (Japan, GPE), (Japan, GPE), (South Korea, GPE), (South Korea, GPE), (Taiwan, GPE), (Taiwan, GPE), (Hong Kong, GPE), (Hong Kong, GPE), (Malaysia, GPE), (Kuala Lumpur, GPE), (Penang, GPE), (Malaysia, GPE), (Vietnam, GPE), (Hanoi, GPE), (Ho Chi Minh City, PERSON), (Mongolia, GPE), (Mongolia, GPE), (Bhutan, GPE), (Gross National Happiness, ORG), (Brunei, GPE), (Brunei, GPE), (Storyteller Travel, ORG), (Nomad, ORG)]",28
2,travel,2,user,What is the most searched topic for those people who decided to travel to Taiwan?,"[(Taiwan, GPE)]",1


In [27]:
spacy.explain("NORP") # To find the explanation of the tag.

'Nationalities or religious or political groups'

In [28]:
trf = pd.read_csv(r"C:\Users\nene0\Downloads\trf_collab.csv")

In [29]:
trf.head()

,Chat_ID,Message_ID,Sender,Message,Spacy_Trf,Spacy_Trf_Count
0,travel,0,user,Can you recommend top 10 places to travel in East Asia where the public security is considered very safe?,"[('10', 'CARDINAL'), ('East Asia', 'LOC')]",2
1,travel,1,copilot,"Sure! Here are ten of the safest places to travel in East Asia, based on public security and overall safety:\n- Singapore: Known for its strict laws and efficient law enforcement, Singapore is often ranked as one of the safest cities in the world.\n- Japan: With low crime rates and a high standard of living, Japan is a top destination for safe travel.\n- South Korea: South Korea boasts a low crime rate and is known for its friendly and helpful locals.\n- Taiwan: Taiwan is praised for its safety, cleanliness, and friendly atmosphere.\n- Hong Kong: Despite recent political unrest, Hong Kong remains relatively safe for tourists, with low violent crime rates.\n- Malaysia: Particularly cities like Kuala Lumpur and Penang, Malaysia is known for its safety and hospitality.\n- Vietnam: Cities like Hanoi and Ho Chi Minh City are generally safe for tourists, with low levels of violent crime.\n- Mongolia: Known for its vast landscapes and nomadic culture, Mongolia is a safe destination with low crime rates.\n- Bhutan: This peaceful country is known for its low crime rates and emphasis on Gross National Happiness.\n- Brunei: With strict laws and a high standard of living, Brunei is considered very safe for travelers.\nThese destinations offer a combination of cultural richness, natural beauty, and a high level of public security. Have you visited any of these places before, or are you planning a trip soon?\n: Storyteller Travel\n: Nomad Capitalist\n: Adventure to Every Country","[('ten', 'CARDINAL'), ('East Asia', 'LOC'), ('Singapore', 'GPE'), ('Singapore', 'GPE'), ('Japan', 'GPE'), ('Japan', 'GPE'), ('South Korea', 'GPE'), ('South Korea', 'GPE'), ('Taiwan', 'GPE'), ('Taiwan', 'GPE'), ('Hong Kong', 'GPE'), ('Hong Kong', 'GPE'), ('Malaysia', 'GPE'), ('Kuala Lumpur', 'GPE'), ('Penang', 'GPE'), ('Malaysia', 'GPE'), ('Vietnam', 'GPE'), ('Hanoi', 'GPE'), ('Ho Chi Minh City', 'GPE'), ('Mongolia', 'GPE'), ('Mongolia', 'GPE'), ('Bhutan', 'GPE'), ('Brunei', 'GPE'), ('Brunei', 'GPE')]",24
2,travel,2,user,What is the most searched topic for those people who decided to travel to Taiwan?,"[('Taiwan', 'GPE')]",1
3,travel,3,copilot,"One of the most searched topics for travelers planning a trip to Taiwan is ""things to know before visiting Taiwan"". This includes practical tips such as booking accommodation early, making restaurant reservations, using the EasyCard for public transportation, and downloading essential transportation apps.\nTravelers are also interested in Taiwan's night markets, street food, and cultural attractions like temples and historical sites. Additionally, many look up information on hiking trails and natural hot springs, as Taiwan is known for its beautiful landscapes and outdoor activities.\nAre you planning a trip to Taiwan, or just curious about what others are interested in?\n: Lonely Planet\n: The Sandy Feet","[('One', 'CARDINAL'), ('Taiwan', 'GPE'), ('Taiwan', 'GPE'), ('EasyCard', 'PRODUCT'), ('Taiwan', 'GPE'), ('Taiwan', 'GPE'), ('Taiwan', 'GPE')]",7
4,travel,4,user,"I am curious to know what other people are interested in. If you are a human, out of those top 10 places, where would you want to go?","[('10', 'CARDINAL')]",1


### Displacy

In [30]:
travel_chat.iloc[0]['Message']

'Can you recommend top 10 places to travel in East Asia where the public security is considered very safe?'

In [31]:
displacy.render(ner(travel_chat.iloc[7]['Message']), style="ent", jupyter=True)

In [32]:
message = travel_chat.iloc[0]['Message']

doc = ner(message) # process message with model

print([token.text for token in doc]) # print the token

['Can', 'you', 'recommend', 'top', '10', 'places', 'to', 'travel', 'in', 'East', 'Asia', 'where', 'the', 'public', 'security', 'is', 'considered', 'very', 'safe', '?']


In [33]:
# Sample text
text = "There was an earthquake in California today."

# Process the text
doc = ner(text)

# Extract named entities
for ent in doc.ents:
    print(ent.text, ent.label_)

California GPE
today DATE


## Flair

In [34]:
# Load the NER tagger
tagger = SequenceTagger.load('ner')

# Function to extract named entities
def extract_entities(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    return [(entity.text, entity.tag) for entity in sentence.get_spans('ner')]


travel_chat.loc[:,'Flair'] = travel_chat['Message'].apply(extract_entities)

2024-09-18 19:36:06,982 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [35]:
travel_chat.loc[:, 'Flair_Count'] = travel_chat['Flair'].apply(len)

In [36]:
travel_chat.head()

,Chat_ID,Message_ID,Sender,Message,Spacy_Lg,Spacy_Lg_Count,Flair,Flair_Count
0,travel,0,user,Can you recommend top 10 places to travel in East Asia where the public security is considered very safe?,"[(10, CARDINAL), (East Asia, LOC)]",2,"[(East Asia, LOC)]",1
1,travel,1,copilot,"Sure! Here are ten of the safest places to travel in East Asia, based on public security and overall safety:\n- Singapore: Known for its strict laws and efficient law enforcement, Singapore is often ranked as one of the safest cities in the world.\n- Japan: With low crime rates and a high standard of living, Japan is a top destination for safe travel.\n- South Korea: South Korea boasts a low crime rate and is known for its friendly and helpful locals.\n- Taiwan: Taiwan is praised for its safety, cleanliness, and friendly atmosphere.\n- Hong Kong: Despite recent political unrest, Hong Kong remains relatively safe for tourists, with low violent crime rates.\n- Malaysia: Particularly cities like Kuala Lumpur and Penang, Malaysia is known for its safety and hospitality.\n- Vietnam: Cities like Hanoi and Ho Chi Minh City are generally safe for tourists, with low levels of violent crime.\n- Mongolia: Known for its vast landscapes and nomadic culture, Mongolia is a safe destination with low crime rates.\n- Bhutan: This peaceful country is known for its low crime rates and emphasis on Gross National Happiness.\n- Brunei: With strict laws and a high standard of living, Brunei is considered very safe for travelers.\nThese destinations offer a combination of cultural richness, natural beauty, and a high level of public security. Have you visited any of these places before, or are you planning a trip soon?\n: Storyteller Travel\n: Nomad Capitalist\n: Adventure to Every Country","[(ten, CARDINAL), (East Asia, LOC), (Singapore, GPE), (Singapore, GPE), (one, CARDINAL), (Japan, GPE), (Japan, GPE), (South Korea, GPE), (South Korea, GPE), (Taiwan, GPE), (Taiwan, GPE), (Hong Kong, GPE), (Hong Kong, GPE), (Malaysia, GPE), (Kuala Lumpur, GPE), (Penang, GPE), (Malaysia, GPE), (Vietnam, GPE), (Hanoi, GPE), (Ho Chi Minh City, PERSON), (Mongolia, GPE), (Mongolia, GPE), (Bhutan, GPE), (Gross National Happiness, ORG), (Brunei, GPE), (Brunei, GPE), (Storyteller Travel, ORG), (Nomad, ORG)]",28,"[(East Asia, LOC), (Singapore, LOC), (Singapore, LOC), (Japan, LOC), (Japan, LOC), (South Korea, LOC), (South Korea, LOC), (Taiwan, LOC), (Taiwan, LOC), (Hong Kong, LOC), (Hong Kong, LOC), (Malaysia, LOC), (Kuala Lumpur, LOC), (Penang, LOC), (Malaysia, LOC), (Vietnam, LOC), (Hanoi, LOC), (Ho Chi Minh City, LOC), (Mongolia, LOC), (Mongolia, LOC), (Bhutan, LOC), (Brunei, LOC), (Brunei, LOC)]",23
2,travel,2,user,What is the most searched topic for those people who decided to travel to Taiwan?,"[(Taiwan, GPE)]",1,"[(Taiwan, LOC)]",1
3,travel,3,copilot,"One of the most searched topics for travelers planning a trip to Taiwan is ""things to know before visiting Taiwan"". This includes practical tips such as booking accommodation early, making restaurant reservations, using the EasyCard for public transportation, and downloading essential transportation apps.\nTravelers are also interested in Taiwan's night markets, street food, and cultural attractions like temples and historical sites. Additionally, many look up information on hiking trails and natural hot springs, as Taiwan is known for its beautiful landscapes and outdoor activities.\nAre you planning a trip to Taiwan, or just curious about what others are interested in?\n: Lonely Planet\n: The Sandy Feet","[(One, CARDINAL), (Taiwan, GPE), (Taiwan, GPE), (Travelers, ORG), (Taiwan, GPE), (Taiwan, GPE), (Taiwan, GPE), (Lonely Planet, WORK_OF_ART), (The Sandy Feet, WORK_OF_ART)]",9,"[(Taiwan, LOC), (Taiwan, LOC), (EasyCard, MISC), (Taiwan, LOC), (Taiwan, LOC), (Taiwan, LOC), (Lonely Planet : The Sandy Feet, MISC)]",7
4,travel,4,user,"I am curious to know what other people are interested in. If you are a human, out of those top 10 places, where wo

In [37]:
travel_chat[['Spacy_Lg', 'Flair']].head(10)

,Spacy_Lg,Flair
0,"[(10, CARDINAL), (East Asia, LOC)]","[(East Asia, LOC)]"
1,"[(ten, CARDINAL), (East Asia, LOC), (Singapore, GPE), (Singapore, GPE), (one, CARDINAL), (Japan, GPE), (Japan, GPE), (South Korea, GPE), (South Korea, GPE), (Taiwan, GPE), (Taiwan, GPE), (Hong Kong, GPE), (Hong Kong, GPE), (Malaysia, GPE), (Kuala Lumpur, GPE), (Penang, GPE), (Malaysia, GPE), (Vietnam, GPE), (Hanoi, GPE), (Ho Chi Minh City, PERSON), (Mongolia, GPE), (Mongolia, GPE), (Bhutan, GPE), (Gross National Happiness, ORG), (Brunei, GPE), (Brunei, GPE), (Storyteller Travel, ORG), (Nomad, ORG)]","[(East Asia, LOC), (Singapore, LOC), (Singapore, LOC), (Japan, LOC), (Japan, LOC), (South Korea, LOC), (South Korea, LOC), (Taiwan, LOC), (Taiwan, LOC), (Hong Kong, LOC), (Hong Kong, LOC), (Malaysia, LOC), (Kuala Lumpur, LOC), (Penang, LOC), (Malaysia, LOC), (Vietnam, LOC), (Hanoi, LOC), (Ho Chi Minh City, LOC), (Mongolia, LOC), (Mongolia, LOC), (Bhutan, LOC), (Brunei, LOC), (Brunei, LOC)]"
2,"[(Taiwan, GPE)]","[(Taiwan, LOC)]"
3,"[(One, CARDINAL), (Taiwan, GPE), (Taiwan, GPE), (Travelers, ORG), (Taiwan, GPE), (Taiwan, GPE), (Taiwan, GPE), (Lonely Planet, WORK_OF_ART), (The Sandy Feet, WORK_OF_ART)]","[(Taiwan, LOC), (Taiwan, LOC), (EasyCard, MISC), (Taiwan, LOC), (Taiwan, LOC), (Taiwan, LOC), (Lonely Planet : The Sandy Feet, MISC)]"
4,"[(10, CARDINAL)]",[]
5,"[(Japan, GPE), (Tokyo, GPE)]","[(Japan, LOC), (Tokyo, LOC)]"
6,"[(Singapore, GPE), (Singapore, GPE), (Japan, GPE), (first, ORDINAL), (the first day, DATE)]","[(Singapore, LOC), (Singapore, LOC), (Japan, LOC)]"
7,"[(Singapore, GPE), (Japan, GPE), (first, ORDINAL), (the first day, DATE), (Asakusa, PERSON), (Tokyo, GPE), (Senso-ji Temple, FAC), (Tokyo, GPE), (Nakamise Street, FAC), (kibi dango, ORG), (Asakusa, PERSON), (Japanese, NORP), (first, ORDINAL), (Singapore, GPE)]","[(Singapore, LOC), (Japan, LOC), (Asakusa, LOC), (Tokyo, LOC), (Senso-ji Temple, LOC), (Tokyo, LOC), (Nakamise Street, LOC), (Asakusa, LOC), (Japanese, MISC), (Singapore, LOC)]"
8,"[(Singaporean, NORP), (laksa, satay, ORG), (bak kut teh, PERSON), (marina, LOC), (10, CARDINAL)]","[(Singaporean, MISC)]"
9,"[(Singapore, GPE), (10, CARDINAL), (Taiwan, GPE), (the first day, DATE), (Taipei, GPE), (Shilin Night Market, FAC), (Taipei, GPE), (Jiufen, ORG)]","[(Singapore, LOC), (Taiwan, LOC), (Taipei, LOC), (Shilin Night Market, LOC), (Taipei, LOC), (Jiufen, LOC)]"


In [40]:
trf_travel = trf[trf['Chat_ID']=='travel']

In [39]:
pd.concat([travel_chat[['Spacy_Lg', 'Flair']], trf_travel[['Spacy_Trf']]], axis=1)

,Spacy_Lg,Flair,Spacy_Trf
0,"[(10, CARDINAL), (East Asia, LOC)]","[(East Asia, LOC)]","[('10', 'CARDINAL'), ('East Asia', 'LOC')]"
1,"[(ten, CARDINAL), (East Asia, LOC), (Singapore, GPE), (Singapore, GPE), (one, CARDINAL), (Japan, GPE), (Japan, GPE), (South Korea, GPE), (South Korea, GPE), (Taiwan, GPE), (Taiwan, GPE), (Hong Kong, GPE), (Hong Kong, GPE), (Malaysia, GPE), (Kuala Lumpur, GPE), (Penang, GPE), (Malaysia, GPE), (Vietnam, GPE), (Hanoi, GPE), (Ho Chi Minh City, PERSON), (Mongolia, GPE), (Mongolia, GPE), (Bhutan, GPE), (Gross National Happiness, ORG), (Brunei, GPE), (Brunei, GPE), (Storyteller Travel, ORG), (Nomad, ORG)]","[(East Asia, LOC), (Singapore, LOC), (Singapore, LOC), (Japan, LOC), (Japan, LOC), (South Korea, LOC), (South Korea, LOC), (Taiwan, LOC), (Taiwan, LOC), (Hong Kong, LOC), (Hong Kong, LOC), (Malaysia, LOC), (Kuala Lumpur, LOC), (Penang, LOC), (Malaysia, LOC), (Vietnam, LOC), (Hanoi, LOC), (Ho Chi Minh City, LOC), (Mongolia, LOC), (Mongolia, LOC), (Bhutan, LOC), (Brunei, LOC), (Brunei, LOC)]","[('ten', 'CARDINAL'), ('East Asia', 'LOC'), ('Singapore', 'GPE'), ('Singapore', 'GPE'), ('Japan', 'GPE'), ('Japan', 'GPE'), ('South Korea', 'GPE'), ('South Korea', 'GPE'), ('Taiwan', 'GPE'), ('Taiwan', 'GPE'), ('Hong Kong', 'GPE'), ('Hong Kong', 'GPE'), ('Malaysia', 'GPE'), ('Kuala Lumpur', 'GPE'), ('Penang', 'GPE'), ('Malaysia', 'GPE'), ('Vietnam', 'GPE'), ('Hanoi', 'GPE'), ('Ho Chi Minh City', 'GPE'), ('Mongolia', 'GPE'), ('Mongolia', 'GPE'), ('Bhutan', 'GPE'), ('Brunei', 'GPE'), ('Brunei', 'GPE')]"
2,"[(Taiwan, GPE)]","[(Taiwan, LOC)]","[('Taiwan', 'GPE')]"
3,"[(One, CARDINAL), (Taiwan, GPE), (Taiwan, GPE), (Travelers, ORG), (Taiwan, GPE), (Taiwan, GPE), (Taiwan, GPE), (Lonely Planet, WORK_OF_ART), (The Sandy Feet, WORK_OF_ART)]","[(Taiwan, LOC), (Taiwan, LOC), (EasyCard, MISC), (Taiwan, LOC), (Taiwan, LOC), (Taiwan, LOC), (Lonely Planet : The Sandy Feet, MISC)]","[('One', 'CARDINAL'), ('Taiwan', 'GPE'), ('Taiwan', 'GPE'), ('EasyCard', 'PRODUCT'), ('Taiwan', 'GPE'), ('Taiwan', 'GPE'), ('Taiwan', 'GPE')]"
4,"[(10, CARDINAL)]",[],"[('10', 'CARDINAL')]"
5,"[(Japan, GPE), (Tokyo, GPE)]","[(Japan, LOC), (Tokyo, LOC)]","[('Japan', 'GPE'), ('Tokyo', 'GPE')]"
6,"[(Singapore, GPE), (Singapore, GPE), (Japan, GPE), (first, ORDINAL), (the first day, DATE)]","[(Singapore, LOC), (Singapore, LOC), (Japan, LOC)]","[('Singapore', 'GPE'), ('Singapore', 'GPE'), ('Japan', 'GPE'), ('first', 'ORDINAL'), ('the first day', 'DATE')]"
7,"[(Singapore, GPE), (Japan, GPE), (first, ORDINAL), (the first day, DATE), (Asakusa, PERSON), (Tokyo, GPE), (Senso-ji Temple, FAC), (Tokyo, GPE), (Nakamise Street, FAC), (kibi dango, ORG), (Asakusa, PERSON), (Japanese, NORP), (first, ORDINAL), (Singapore, GPE)]","[(Singapore, LOC), (Japan, LOC), (Asakusa, LOC), (Tokyo, LOC), (Senso-ji Temple, LOC), (Tokyo, LOC), (Nakamise Street, LOC), (Asakusa, LOC), (Japanese, MISC), (Singapore, LOC)]","[('Singapore', 'GPE'), ('Japan', 'GPE'), ('the first day', 'DATE'), ('Asakusa', 'FAC'), ('Tokyo', 'GPE'), ('Senso-ji Temple', 'FAC'), ('Tokyo', 'GPE'), ('Nakamise Street', 'FAC'), ('Asakusa', 'FAC'), ('Japanese', 'NORP'), ('first', 'ORDINAL'), ('Singapore', 'GPE')]"
8,"[(Singaporean, NORP), (laksa, satay, ORG), (bak kut teh, PERSON), (marina, LOC), (10, CARDINAL)]","[(Singaporean, MISC)]","[('Singaporean', 'NORP'), ('first', 'ORDINAL'), ('marina bay', 'FAC'), ('10', 'CARDINAL')]"
9,"[(Singapore, GPE), (10, CARDINAL), (Taiwan, GPE), (the first day, DATE), (Taipei, GPE), (Shilin Night Market, FAC), (Taipei, GPE), (Jiufen, ORG)]","[(Singapore, LOC), (Taiwan, LOC), (Taipei, LOC), (Shilin Night Market, LOC), (Taipei, LOC), (Jiufen, LOC)]","[('Singapore', 'GPE'), ('10', 'CARDINAL'), ('Taiwan', 'GPE'), ('the first day', 'DATE'), ('Taipei', 'GPE'), ('Shilin Night Market', 'FAC'), ('Taipei 101', 'FAC'), ('Jiufen', 'GPE')]"


In [25]:
spacy.explain("FAC")

'Buildings, airports, highways, bridges, etc.'

In [20]:
travel_chat.iloc[7]['Message']

"Singapore sounds like a fantastic choice, especially with its amazing food scene! Revisiting as an adult will definitely give you a whole new perspective.\nIf I could travel to Japan, the first thing I'd want to do on the first day is visit Asakusa in Tokyo. It's home to the famous Senso-ji Temple, Tokyo's oldest temple, which offers a beautiful blend of history and culture. The area around the temple, known as Nakamise Street, is filled with traditional shops and street food stalls where you can try local snacks like taiyaki (fish-shaped cakes filled with sweet red bean paste) and kibi dango (sweet rice dumplings).\nAfter exploring Asakusa, I'd probably head to a nearby onsen (hot spring) to relax and unwind from the journey. It's a perfect way to experience a traditional Japanese bath and soak in the local culture.\nWhat about you? What's the first thing you'd like to do when you visit Singapore again?"